<a href="https://colab.research.google.com/github/imran1386khan/Biobert-Training-Google-Colab/blob/master/WebMD_DataScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Selenium Starts here

import pandas as pd
import numpy as np
import os

!apt-get update

!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

from selenium.webdriver.common.by import By

wd = webdriver.Chrome('chromedriver',options=options)

file_name = "/content/drive/My Drive/Web_md_data/67_re.csv"
qa_pair = {}

for idx in range(67,68):
  wd.get("https://www.webmd.com/a-to-z-guides/qa")
  for link in wd.find_elements_by_xpath('//*[@id="ContentPane32"]/div/div/div/ul/li['+ str(idx) +']/a'):
    base_link = link.get_attribute('href')
    if base_link != 'https://qa/default.htm':
      print('BASE LINK : ',base_link)
      for pg_num in range(100):
        try:
          wd.get(base_link + "?pg=" + str(pg_num)) # wget change to move to next page
          for q_num in range(30):
            wd.get(base_link + "?pg=" + str(pg_num)) # wget change for getting next question on the page
            for question in wd.find_elements_by_xpath('//*[@id="ContentPane31"]/div/div[2]/ul/li[' + str(q_num) + ']/a'):
              que = question.get_attribute('href')
              if len(que) > 5:
                print('Question post lenth check > 5: ', que)
                wd.get(que) # wget chnage to capture new 'que' and its corresponding 'ans'
                question_obj = wd.find_element_by_xpath('//*[@id="ContentPane31"]/div/div[3]/div/h1')
                answer_obj = wd.find_element_by_xpath('//*[@id="art-ans"]')
                answer_text = answer_obj.text.replace('\n', ',')
                question_text = question_obj.text
                # print(question_text)
                # print(answer_text)
                qa_pair.update({question_text : answer_text})
            print('----question number {} on page {}----:'.format(q_num,pg_num))
          print('----Moving to next page number : ----:', pg_num)
          df = pd.DataFrame.from_dict(qa_pair, orient='index')
          print('Creating DF')
          df.reset_index(level=0, inplace=True)
          df.rename(columns={'index':'Question', 0:'Answer'}, inplace =True)
          if not os.path.isfile(file_name):
            df.to_csv(file_name, index=False)
            # print('Saving DF')
          else:
            df.to_csv(file_name, mode='a', index=False)
        except:
              pass
